In [2]:
import pandas as pd
from ufal.morphodita import *
import csv
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
data = pd.read_csv('articles_cleaned.csv',  sep='|')
data.columns = ['id','query','url', 'headline', 'paragraphs']
data['paragraphs'] = data['paragraphs'].fillna('')

In [6]:
data =  data[data['paragraphs'] != '\xa0']  

In [7]:
lemma_column = data.shape[0]*[[]]

In [8]:
data['lemmas'] = lemma_column

In [11]:
tagger = Tagger.load('../../morphodita/czech-morfflex-pdt-161115.tagger')
forms = Forms()

In [ ]:
for index, row in data.iterrows():
    paragraph = row['paragraphs']
    lemmas = TaggedLemmas()
    tokens = TokenRanges()
    tokenizer = tagger.newTokenizer()
    tokenizer.setText(paragraph)
    allLemmas = []
    while tokenizer.nextSentence(forms, tokens):
        newLemmas = []
        tagger.tag(forms, lemmas)
        newLemmas = [l.lemma for l in lemmas]
        allLemmas.extend(newLemmas)
#     print(allLemmas)
    row['lemmas'] = allLemmas

In [80]:
lemma_data = data['lemmas']

In [81]:
lemma_data

0     [mnohovrstevnatý, ,, sebevědomý, a-1, úspěšný,...
1     [komunista, předběhnout_:W, sociální, demokrac...
2     [bývalý_^(*2t), pražský, imám, Samer_;S, Sheha...
4     [toupat_:T, na-1, vrchol, vodárenský, věž, ,, ...
5     [případ, rakouský, penzionovaný_^(*2t), plukov...
6     [Stanislav_;Y, Chvála_;S, (, 38, ), být, gener...
7     [v-1, střet, mezi-1, vedení_^(*5ést), ČEZ-1_:B...
8     [byznysový, model, klasický, personální, agent...
9     [vedení_^(*5ést), polostátní, firma, ČEZ-1_:B_...
10    [mobilní, operátor, T-Mobile_;K, zvýšit_:W, od...
11    [v-1, čelo, automobilka, Tesla-1_;S, nahradit_...
12    [komerční, banka, za, tři`3, čtvrtletí, letošn...
13    [od-1, mezinárodní, konference, o-1, pornobyzn...
15    [český, vláda, moci_^(mít_možnost_[něco_dělat]...
16    [klient, od-1, banka, požadovat_:T, tři`3, zák...
17    [od-1, mezinárodní, konference, o-1, pornobyzn...
18    [nabídka, měsíc, zdarma, platit_:T, pouze, pro...
19    [když, se_^(zvr._zájmeno/částice), říci, v

In [82]:
dictionary = gensim.corpora.Dictionary(lemma_data)
dictionary.filter_extremes(no_below=1, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in lemma_data]

In [87]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=3, id2word=dictionary, passes=2, workers=2)

In [88]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.006*"tento" + 0.004*"tak-3" + 0.004*"těžba" + 0.004*"správce" + 0.004*"chtít" + 0.004*"firma" + 0.004*"po-1" + 0.003*"karta" + 0.003*"zpracování_^(*3at)" + 0.003*"předplatné_^(poplatek)"
Topic: 1 Word: 0.017*""" + 0.005*"říci" + 0.004*"člověk" + 0.004*"když" + 0.004*"jít" + 0.004*"podle-2" + 0.004*"všechen" + 0.004*"velký" + 0.004*"tak-3" + 0.003*"evropský"
Topic: 2 Word: 0.009*""" + 0.006*"procento" + 0.004*"svůj-1_^(přivlast.)" + 0.004*"koruna" + 0.004*"banka" + 0.003*"architektura" + 0.003*"Praha_;G" + 0.003*"tento" + 0.003*"dům" + 0.003*"informace"
